In [1]:
# Import all relevant libraries for cleaning data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Importing data

train=pd.read_csv('data/cs-training.csv',index_col=0)
train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [3]:
train.columns

Index(['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

So far, we can keep all columns

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

In [5]:
# Let's see if we have any null values

pd.DataFrame({'Number of Null Values':train.isnull().sum(),'Ratio':train.isnull().sum()
             /len(train)*100})

,Number of Null Values,Ratio
SeriousDlqin2yrs,0,0.000000
RevolvingUtilizationOfUnsecuredLines,0,0.000000
age,0,0.000000
NumberOfTime30-59DaysPastDueNotWorse,0,0.000000
DebtRatio,0,0.000000
MonthlyIncome,29731,19.820667
NumberOfOpenCreditLinesAndLoans,0,0.000000
NumberOfTimes90DaysLate,0,0.000000
NumberRealEstateLoansOrLines,0,0.000000
NumberOfTime60-89DaysPastDueNotWorse,0,0.000000


There is 19.82% null values for Monthly Income and 2.6% null values for Number of Dependents.

Let's first analyze the null values for Monthly Income.

In [6]:
# Comparing values for data where Monthly Income is null vs not null to see what to do with
# these values

monthly_income_null= train[train['MonthlyIncome'].isnull()]
monthly_income_not_null=train[train['MonthlyIncome'].notnull()]

In [7]:
for i in train.columns:
    print([i])
    print(pd.DataFrame({'MonthlyIncomeNull': monthly_income_null[i].describe(),
                 'MonthlyIncomeNotNull': monthly_income_not_null[i].describe()}))

['SeriousDlqin2yrs']
       MonthlyIncomeNull  MonthlyIncomeNotNull
count       29731.000000         120269.000000
mean            0.056137              0.069486
std             0.230189              0.254280
min             0.000000              0.000000
25%             0.000000              0.000000
50%             0.000000              0.000000
75%             0.000000              0.000000
max             1.000000              1.000000
['RevolvingUtilizationOfUnsecuredLines']
       MonthlyIncomeNull  MonthlyIncomeNotNull
count       29731.000000         120269.000000
mean            6.649421              5.899873
std           217.814854            257.040685
min             0.000000              0.000000
25%             0.016027              0.035084
50%             0.081697              0.177282
75%             0.440549              0.579428
max         22198.000000          50708.000000
['age']
       MonthlyIncomeNull  MonthlyIncomeNotNull
count       29731.000000         1202

The main concerning thing from this comparison, is the difference in Debt Ratio where the data where monthly income is null has a mean of 1673 meaning that they have 1673 times more debt than their monthly income which is really concerning.

The data where monthly income is not null has a mean of 26 which is much more reasonable. 

This could potentially be a data error because it seems unreasonable that while on average they have 1673 times more debt than income, they have a lower average delinquent rate(0.56 vs 0.069)

Let's look at this more closely

In [8]:
train['DebtRatio'].quantile([.975])

0.975    3489.025
Name: DebtRatio, dtype: float64

In [9]:
train[train['DebtRatio']>3489.025][['SeriousDlqin2yrs','MonthlyIncome']].describe()

,SeriousDlqin2yrs,MonthlyIncome
count,3750.000000,185.000000
mean,0.064267,0.064865
std,0.245260,0.246956
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


This is very concerning. Out of around 4000 records with DebtRatio >3489, only 185 have a value for MonthlyIncome. Further, out of the ones that do have monthly income, the average is 0.06 and similar to the mean of deliquency which means that this is probably due to a data entry error. 

In addition, the mean deliquency of these people with Debt Ratio >3489 is about the same as the average deliquency, which doesn't make sense.

Therefore we will remove all data where the DebtRatio is over 3489.025

In [10]:
# Dropping DebtRatio outliers

train= train[train['DebtRatio']<3489.025]

In [11]:
train['DebtRatio'].describe()

count    146250.000000
mean        210.326026
std         620.473755
min           0.000000
25%           0.169896
50%           0.355985
75%           0.757884
max        3489.000000
Name: DebtRatio, dtype: float64

In [12]:
train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [13]:
train.isnull().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           26166
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3694
dtype: int64

For the rest of the null values, I will impute by doing the following:

- Identify the column that is highest correlated with the column that has null values
- Group the values in that correlated column into a few buckets
- Make a dataframe for each bucket and find the mean value for the column with null values in that bucket
- Fill in the missing values with the mean for each bucket.
- Concatanate



In [14]:
# Impute missing values for MonthlyIncome

train[train.columns[0:]].corr()['MonthlyIncome']

SeriousDlqin2yrs                       -0.019762
RevolvingUtilizationOfUnsecuredLines    0.007110
age                                     0.037724
NumberOfTime30-59DaysPastDueNotWorse   -0.010237
DebtRatio                              -0.043272
MonthlyIncome                           1.000000
NumberOfOpenCreditLinesAndLoans         0.091866
NumberOfTimes90DaysLate                -0.012766
NumberRealEstateLoansOrLines            0.125703
NumberOfTime60-89DaysPastDueNotWorse   -0.011140
NumberOfDependents                      0.062974
Name: MonthlyIncome, dtype: float64

Monthly Income is most correlated to NumberRealEstateLoansorLines.

In [15]:
train['NumberRealEstateLoansOrLines'].value_counts()

0     56147
1     51392
2     29895
3      5662
4      1884
5       592
6       283
7       141
8        78
9        63
10       33
12       17
11       16
13       14
14        7
15        5
16        4
17        3
25        3
19        2
20        2
54        1
21        1
18        1
26        1
23        1
29        1
32        1
Name: NumberRealEstateLoansOrLines, dtype: int64

In [16]:
frames=[]
for i in train['NumberRealEstateLoansOrLines'].unique():
    df_cat=train[train['NumberRealEstateLoansOrLines']== i]
    if len(df_cat)>1:
        df_cat['MonthlyIncome'].fillna(df_cat['MonthlyIncome'].mean(),inplace=True)
    else:
        df_cat['MonthlyIncome'].fillna(train['MonthlyIncome'].mean(),inplace=True)
    frames.append(df_cat)
    train_impute_income=pd.concat(frames)
    

/Users/felixinoue/opt/anaconda3/envs/smartpay/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [17]:
train_impute_income.isnull().sum()

SeriousDlqin2yrs                           0
RevolvingUtilizationOfUnsecuredLines       0
age                                        0
NumberOfTime30-59DaysPastDueNotWorse       0
DebtRatio                                  0
MonthlyIncome                              0
NumberOfOpenCreditLinesAndLoans            0
NumberOfTimes90DaysLate                    0
NumberRealEstateLoansOrLines               0
NumberOfTime60-89DaysPastDueNotWorse       0
NumberOfDependents                      3694
dtype: int64

In [18]:
# Impute missing values for Number of Dependents

train_impute_income[train_impute_income.columns[0:]].corr()['NumberOfDependents']

SeriousDlqin2yrs                        0.047300
RevolvingUtilizationOfUnsecuredLines    0.001896
age                                    -0.213909
NumberOfTime30-59DaysPastDueNotWorse   -0.002835
DebtRatio                              -0.092878
MonthlyIncome                           0.063482
NumberOfOpenCreditLinesAndLoans         0.068932
NumberOfTimes90DaysLate                -0.010454
NumberRealEstateLoansOrLines            0.136602
NumberOfTime60-89DaysPastDueNotWorse   -0.011247
NumberOfDependents                      1.000000
Name: NumberOfDependents, dtype: float64

Age is most correlated with Number of Dependents.

In [19]:
train_impute_income['age'].value_counts()

49     3725
48     3657
50     3653
46     3594
47     3592
       ... 
102       3
109       2
105       1
107       1
0         1
Name: age, Length: 86, dtype: int64

In [20]:
frames=[]
for i in train_impute_income['age'].unique():
    df_cat=train_impute_income[train_impute_income['age']==i]
    if len(df_cat)>2:
        df_cat['NumberOfDependents'].fillna(df_cat['NumberOfDependents'].median(),inplace=True)
    else:
        df_cat['NumberOfDependents'].fillna(train_impute_income['NumberOfDependents'].median(),inplace=True)
    frames.append(df_cat)
    train_new=pd.concat(frames)

In [21]:
train_new.isnull().sum()

SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

Cool, now we have no missing values.

Now let's see if we have any duplicate values.

In [30]:
train_new.duplicated().sum()

754

In [32]:
# Removing the duplicates

train_new.drop_duplicates(inplace=True)

# Checking if we still have duplicates

train_new.duplicated().sum()

0

In [35]:
train_new.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
6185,0,0.155932,45,0,1.186438,6650.0,20,0,6,0,2.0
15458,0,0.039168,45,0,1.007399,5000.0,17,0,6,0,3.0
40153,0,0.145135,45,0,0.587821,32366.0,12,0,6,0,3.0
57864,0,0.221642,45,0,0.504239,13800.0,14,0,6,0,0.0


In [36]:
# Let's round columns to two decimal places

train_new['RevolvingUtilizationOfUnsecuredLines']=round(train_new['RevolvingUtilizationOfUnsecuredLines'],2)
train_new['DebtRatio']=round(train_new['DebtRatio'],2)
train_new.head()


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.77,45,2,0.80,9120.0,13,0,6,0,2.0
6185,0,0.16,45,0,1.19,6650.0,20,0,6,0,2.0
15458,0,0.04,45,0,1.01,5000.0,17,0,6,0,3.0
40153,0,0.15,45,0,0.59,32366.0,12,0,6,0,3.0
57864,0,0.22,45,0,0.50,13800.0,14,0,6,0,0.0


This concludes the data cleaning part of this project. 

In [37]:
train_new.to_csv('data_cleaned.csv')